# AI vs Human Art ML Project
### Step 1: Data Loading

We still need to check if this is the correct way to load the data..(we can ask professor if this is correct)
Also you need to load the data from your own machine after you download it(having it on git is not good apparently)


In [1]:
# Imports
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Loading the data
train_set = tf.keras.preprocessing.image_dataset_from_directory(
    "data/train",
    validation_split=0.2,
    subset="training",
    seed=123,
    label_mode = "binary",
    image_size=(512, 512), # All the images in the dataset are this size
    batch_size=32,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data/train",
    validation_split=0.2,
    subset="validation",
    seed=123,
    label_mode = "binary",
    image_size=(512, 512),
    batch_size=32,
)

# Step 2: EDA

print("heelo")

In [2]:
print("hello")

hello
